In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models.detection as detection
from ultralytics import YOLO
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection

# Set dataset paths
data_dir = "path/to/dataset"  # Ganti dengan path dataset COCO
annotation_file = "path/to/annotations.json"  # Ganti dengan path anotasi COCO

# Transformasi data
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load dataset
dataset = CocoDetection(root=data_dir, annFile=annotation_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Load Faster R-CNN model
faster_rcnn = detection.fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn.train()

# Set optimizer dan loss function
optimizer = torch.optim.Adam(faster_rcnn.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

def train_faster_rcnn(model, dataloader, epochs=10):
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for images, targets in dataloader:
            images = [image for image in images]
            targets = [{k: v for k, v in t.items()} for t in targets]
            
            optimizer.zero_grad()
            loss_dict = model(images, targets)
            loss = sum(loss for loss in loss_dict.values())
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        losses.append(epoch_loss / len(dataloader))
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(dataloader):.4f}")
    return losses

# Train Faster R-CNN
faster_rcnn_losses = train_faster_rcnn(faster_rcnn, dataloader)

# Load YOLOv8 model
yolo_model = YOLO('best.pt')

# Train YOLOv8
yolo_results = yolo_model.train(data='path/to/yolo_dataset.yaml', epochs=10, imgsz=640)

# Plot loss comparison
plt.plot(range(1, 11), faster_rcnn_losses, label='Faster R-CNN Loss')
plt.plot(range(1, 11), yolo_results['train_loss'], label='YOLOv8 Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Comparison")
plt.legend()
plt.show()
